In [9]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.stattools import kpss
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.graphics.tsaplots import plot_acf
from pylab import rcParams
from IPython.display import display
import math

import warnings
from statsmodels.tools.sm_exceptions import InterpolationWarning
warnings.simplefilter("ignore", category=InterpolationWarning)
# pd.set_option('future.no_silent_downcasting', False)


#### Data preparation

In [21]:
# open and read excel file
df1 = pd.read_excel("C:/Users/Acer/Desktop/Madi/request35623.xlsx", sheet_name = 'Лист1')
df1.head(5)
print(df1.shape)

c:\Users\Acer\AppData\Local\Programs\Python\Python312\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


(65001, 22)


In [24]:
# Set the first row as column labels
df1.columns = df1.iloc[0]  # Assign first row as header
df_list1 = df1[1:].reset_index(drop=True)  # Drop the first row and reset index
df_list1.head(5)

,БИН,Толық атауы,Полное наименование,Дата регистрации,ОКЭД,Негізгі қызмет түрінің атауы,Наименование основного вида деятельности,Втор.ОКЭД,КРП,ККЖ атауы,...,МНЖ атауы,Наименование КФС,КСЭ новый,ЭСЖ атауы,Наименование КСЭ,КАТО,Елді мекеннің атауы,Наименование населенного пункта,"Заңды мекен-жайы, Юридический адрес","Басшының ТАӘ, ФИО руководителя"
0,000140000012,"""ЖАМБЫЛ ОБЛЫСЫ ӘКІМДІГІНІҢ ҚАРЖЫ БАСҚАРМАСЫ"" К...","КОММУНАЛЬНОЕ ГОСУДАРСТВЕННОЕ УЧРЕЖДЕНИЕ ""УПРАВ...",2005-01-14 00:00:00,84112,Өңірлік басқару органдарының қызметі,Деятельность региональных органов управления,NaN,160,Шағын кәсіпорындар (51-100),...,Коммуналдық меншік,Коммунальная собственность,1321,"Жергілікті мемлекттік басқару органдары, әлеум...","Органы местного государственного управления, к...",311011100,"ЖАМБЫЛ ОБЛЫСЫ, ТАРАЗ Қ.Ә., ТАРАЗ Қ., ӘУЛИЕАТА ...","ЖАМБЫЛСКАЯ ОБЛАСТЬ, ТАРАЗ Г.А., Г.ТАРАЗ, РАЙОН...","улица Желтоксан, здание 78",ӘКЕК БАТЫРХАН БАЗАРБАЙҰЛЫ
1,000140000032,"""ПРОФИЛЬ"" ЖАУАПКЕРШІЛІГІ ШЕКТЕУЛІ СЕРІКТЕСТІГІ","ТОВАРИЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""...",2000-01-17 00:00:00,11070,"Алкогольсіз сусындарды, минералды суларды және...","Производство безалкогольных напитков, минераль...",NaN,105,Шағын кәсіпорындар (<= 5),...,Кәсіпорындардың мемлекет пен шет ел қатыспағ...,Собственность предприятий без государственного...,1122,Ұлттық жеке қаржылық емес корпорациялар – ПАҰ,Национальные частные нефинансовые корпорации –...,271010000,"БАТЫС ҚАЗАҚСТАН ОБЛЫСЫ, ОРАЛ Қ.Ә., ОРАЛ Қ.","ЗАПАДНО-КАЗАХСТАНСКАЯ ОБЛАСТЬ, УРАЛЬСК Г.А., Г...","УЛИЦА НЕКРАСОВА, 29Б",АЛИЕВА НАТАЛИЯ КОНСТАНТИНОВНА
2,000140000042,"""БРОКЕР-СЕРВИС"" ЖАУАПКЕРШІЛІГІ ШЕКТЕУЛІ СЕРІКТ...","ТОВАРИЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""...",2000-01-14 00:00:00,46462,Тегін медициналық көмек көрсету кепілдік көлем...,Оптовая реализация фармацевтических и медицинс...,NaN,105,Шағын кәсіпорындар (<= 5),...,Кәсіпорындардың мемлекет пен шет ел қатыспағ...,Собственность предприятий без государственного...,1122,Ұлттық жеке қаржылық емес корпорациялар – ПАҰ,Национальные частные нефинансовые корпорации –...,311013100,"ЖАМБЫЛ ОБЛЫСЫ, ТАРАЗ Қ.Ә., ТАРАЗ Қ., ЖІБЕК ЖОЛ...","ЖАМБЫЛСКАЯ ОБЛАСТЬ, ТАРАЗ Г.А., Г.ТАРАЗ, РАЙОН...","улица Байзак батыра, строение 247В",ЗУБКО ЮЛИЯ ВАЛЕРЬЕВНА
3,000140000062,"""БАРЫС СТРОЙ ЛТД"" ЖАУАПКЕРШІЛІГІ ШЕКТЕУЛІ СЕРІ...","ТОВАРИЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""...",2000-01-14 00:00:00,41202,"1, 2-санаттағы стационарлық сауда объектілерін...","Строительство нежилых зданий, за исключением с...",NaN,105,Шағын кәсіпорындар (<= 5),...,Кәсіпорындардың мемлекет пен шет ел қатыспағ...,Собственность предприятий без государственного...,1122,Ұлттық жеке қаржылық емес корпорациялар – ПАҰ,Национальные частные нефинансовые корпорации –...,751310000,"АЛМАТЫ ҚАЛАСЫ, ӘУЕЗОВ АУДАНЫ","Г.АЛМАТЫ, АУЭЗОВСКИЙ РАЙОН","Микрорайон Аксай-1а, дом 28Б, кв. 4",ИМАНБЕКОВА САНДУГАШ АМАНТАЕВНА
4,000140000072,"""РТС ДЕКО"" АКЦИОНЕРЛІК ҚОҒАМЫ","АКЦИОНЕРНОЕ ОБЩЕСТВО ""РТС ДЕКО""",2000-01-31 00:00:00,73110,Жарнама агенттіктерінің қызметі,Деятельность рекламных агентств,NaN,160,Шағын кәсіпорындар (51-100),...,Бірлескен кәсіпорындардың шет ел қатысқан менш...,Собственность совместных предприятий с иностра...,1122,Ұлттық жеке қаржылық емес корпорациялар – ПАҰ,Национальные частные нефинансовые корпорации –...,751410000,"АЛМАТЫ ҚАЛАСЫ, БОСТАНДЫҚ АУДАНЫ","Г.АЛМАТЫ, БОСТАНДЫКСКИЙ РАЙОН","УЛ ЕГИЗБАЕВА, 52",КАГАРОВ АРМАН ГАЙРАТОВИЧ


In [16]:
# remove first two rows (blank)
df = df.drop(index=df.index[:2]).reset_index(drop=True)
df.head(5)

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 113,Unnamed: 114,Unnamed: 115,Unnamed: 116,Unnamed: 117,Unnamed: 118,Unnamed: 119,Unnamed: 120,Unnamed: 121,Unnamed: 122
0,Показатель,Категория хозяйства,Регион,2015-01-01 00:00:00,2015-02-01 00:00:00,2015-03-01 00:00:00,2015-04-01 00:00:00,2015-05-01 00:00:00,2015-06-01 00:00:00,2015-07-01 00:00:00,...,2024-03-01 00:00:00,2024-04-01 00:00:00,2024-05-01 00:00:00,2024-06-01 00:00:00,2024-07-01 00:00:00,2024-08-01 00:00:00,2024-09-01 00:00:00,2024-10-01 00:00:00,2024-11-01 00:00:00,2024-12-01 00:00:00
1,КРС,Сельхозпредприятия,РЕСПУБЛИКА КАЗАХСТАН,3400.96,3685.9,4154.43,5203.94,5084.05,6718.83,4235.03,...,9703.45,11165.56,8179.14,24407.52,9835.22,11955.41,16955.56,10638.42,9612.92,21521.5
2,КРС,Сельхозпредприятия,АКМОЛИНСКАЯ ОБЛАСТЬ,514.75,497.95,863.93,667.81,633.39,600.86,875.47,...,978.41,1071.35,1067.76,1119.61,898.07,1382,1133.36,1073.27,1118.19,1332.88
3,КРС,Сельхозпредприятия,АКТЮБИНСКАЯ ОБЛАСТЬ,554.41,504.35,493.56,445.51,585.79,316.47,469.49,...,1520.51,1747.09,915.8,1851.96,932.63,945.11,1142.92,1637.64,1180.54,1045.69
4,КРС,Сельхозпредприятия,АЛМАТИНСКАЯ ОБЛАСТЬ,178.3,362.86,352.49,451.79,228.32,708.84,280.55,...,1055.43,110.45,188.15,172.63,112.31,134.99,1212.02,260.02,129.38,159.66


In [17]:
# Set the first row as column labels
df.columns = df.iloc[0]  # Assign first row as header
df = df[1:].reset_index(drop=True)  # Drop the first row and reset index
df.head(5)

,Показатель,Категория хозяйства,Регион,2015-01-01 00:00:00,2015-02-01 00:00:00,2015-03-01 00:00:00,2015-04-01 00:00:00,2015-05-01 00:00:00,2015-06-01 00:00:00,2015-07-01 00:00:00,...,2024-03-01 00:00:00,2024-04-01 00:00:00,2024-05-01 00:00:00,2024-06-01 00:00:00,2024-07-01 00:00:00,2024-08-01 00:00:00,2024-09-01 00:00:00,2024-10-01 00:00:00,2024-11-01 00:00:00,2024-12-01 00:00:00
0,КРС,Сельхозпредприятия,РЕСПУБЛИКА КАЗАХСТАН,3400.96,3685.9,4154.43,5203.94,5084.05,6718.83,4235.03,...,9703.45,11165.56,8179.14,24407.52,9835.22,11955.41,16955.56,10638.42,9612.92,21521.5
1,КРС,Сельхозпредприятия,АКМОЛИНСКАЯ ОБЛАСТЬ,514.75,497.95,863.93,667.81,633.39,600.86,875.47,...,978.41,1071.35,1067.76,1119.61,898.07,1382,1133.36,1073.27,1118.19,1332.88
2,КРС,Сельхозпредприятия,АКТЮБИНСКАЯ ОБЛАСТЬ,554.41,504.35,493.56,445.51,585.79,316.47,469.49,...,1520.51,1747.09,915.8,1851.96,932.63,945.11,1142.92,1637.64,1180.54,1045.69
3,КРС,Сельхозпредприятия,АЛМАТИНСКАЯ ОБЛАСТЬ,178.3,362.86,352.49,451.79,228.32,708.84,280.55,...,1055.43,110.45,188.15,172.63,112.31,134.99,1212.02,260.02,129.38,159.66
4,КРС,Сельхозпредприятия,АТЫРАУСКАЯ ОБЛАСТЬ,20.8,0.8,15.28,1.97,2.04,1.32,0.8,...,13.52,25.1,13.7,16.08,17.35,41.57,56.59,49.5,40.76,27.22


In [18]:
# convert date columns to datetime format
df.columns = list(df.columns[:3]) + list(pd.to_datetime(df.columns[-120:]))
df = df.rename(columns={col: col.strftime("%Y-%m") for col in df.columns[-120:]}) # change "2015-01-01 00:00:00" to "2015-01" 
df.head(5)

,Показатель,Категория хозяйства,Регион,2015-01,2015-02,2015-03,2015-04,2015-05,2015-06,2015-07,...,2024-03,2024-04,2024-05,2024-06,2024-07,2024-08,2024-09,2024-10,2024-11,2024-12
0,КРС,Сельхозпредприятия,РЕСПУБЛИКА КАЗАХСТАН,3400.96,3685.9,4154.43,5203.94,5084.05,6718.83,4235.03,...,9703.45,11165.56,8179.14,24407.52,9835.22,11955.41,16955.56,10638.42,9612.92,21521.5
1,КРС,Сельхозпредприятия,АКМОЛИНСКАЯ ОБЛАСТЬ,514.75,497.95,863.93,667.81,633.39,600.86,875.47,...,978.41,1071.35,1067.76,1119.61,898.07,1382,1133.36,1073.27,1118.19,1332.88
2,КРС,Сельхозпредприятия,АКТЮБИНСКАЯ ОБЛАСТЬ,554.41,504.35,493.56,445.51,585.79,316.47,469.49,...,1520.51,1747.09,915.8,1851.96,932.63,945.11,1142.92,1637.64,1180.54,1045.69
3,КРС,Сельхозпредприятия,АЛМАТИНСКАЯ ОБЛАСТЬ,178.3,362.86,352.49,451.79,228.32,708.84,280.55,...,1055.43,110.45,188.15,172.63,112.31,134.99,1212.02,260.02,129.38,159.66
4,КРС,Сельхозпредприятия,АТЫРАУСКАЯ ОБЛАСТЬ,20.8,0.8,15.28,1.97,2.04,1.32,0.8,...,13.52,25.1,13.7,16.08,17.35,41.57,56.59,49.5,40.76,27.22


In [19]:
# check values
print(df["Показатель"].unique())
print(df["Категория хозяйства"].unique())
print(df["Регион"].unique())

['КРС' 'Овцы и козы' 'Свиньи' 'Лошади' 'Верблюды' 'Птицы' 'Молоко' 'Яйца']
['Сельхозпредприятия' 'Крестьянские или фермерские хозяйства'
 'Хозяйства населения']
['РЕСПУБЛИКА КАЗАХСТАН' '    АКМОЛИНСКАЯ ОБЛАСТЬ'
 '    АКТЮБИНСКАЯ ОБЛАСТЬ' '    АЛМАТИНСКАЯ ОБЛАСТЬ'
 '    АТЫРАУСКАЯ ОБЛАСТЬ' '    ЗАПАДНО0КАЗАХСТАНСКАЯ ОБЛАСТЬ'
 '    ЖАМБЫЛСКАЯ ОБЛАСТЬ' '    КАРАГАНДИНСКАЯ ОБЛАСТЬ'
 '    КОСТАНАЙСКАЯ ОБЛАСТЬ' '    КЫЗЫЛОРДИНСКАЯ ОБЛАСТЬ'
 '    МАНГИСТАУСКАЯ ОБЛАСТЬ' '    ЮЖНО0КАЗАХСТАНСКАЯ ОБЛАСТЬ'
 '    ПАВЛОДАРСКАЯ ОБЛАСТЬ' '    СЕВЕРО0КАЗАХСТАНСКАЯ ОБЛАСТЬ'
 '    ВОСТОЧНО0КАЗАХСТАНСКАЯ ОБЛАСТЬ' '    ГАСТАНА' '    ГАЛМАТЫ'
 '    ГШЫМКЕНТ' '    ТУРКЕСТАНСКАЯ ОБЛАСТЬ' '    ОБЛАСТЬ ЖЕТІСУ'
 '    ОБЛАСТЬ ҰЛЫТАУ' '    ОБЛАСТЬ АБАЙ']


In [20]:
df["Регион"] = df["Регион"].str.strip() # strip unnecessary spaces

rename_dict = {
    "ВОСТОЧНО0КАЗАХСТАНСКАЯ ОБЛАСТЬ": "ВОСТОЧНО-КАЗАХСТАНСКАЯ ОБЛАСТЬ", 
    "ГАЛМАТЫ": "ГОРОД АЛМАТЫ",
    "ГАСТАНА": "ГОРОД АСТАНА",
    "ГШЫМКЕНТ": "ГОРОД ШЫМКЕНТ",  
    "ЗАПАДНО0КАЗАХСТАНСКАЯ ОБЛАСТЬ": "ЗАПАДНО-КАЗАХСТАНСКАЯ ОБЛАСТЬ",
    "СЕВЕРО0КАЗАХСТАНСКАЯ ОБЛАСТЬ": "СЕВЕРО-КАЗАХСТАНСКАЯ ОБЛАСТЬ",
    "ЮЖНО0КАЗАХСТАНСКАЯ ОБЛАСТЬ": "ЮЖНО-КАЗАХСТАНСКАЯ ОБЛАСТЬ",  
}

# Rename region names 
df["Регион"] = df["Регион"].replace(rename_dict)

df["Регион"].unique()

array(['РЕСПУБЛИКА КАЗАХСТАН', 'АКМОЛИНСКАЯ ОБЛАСТЬ',
       'АКТЮБИНСКАЯ ОБЛАСТЬ', 'АЛМАТИНСКАЯ ОБЛАСТЬ', 'АТЫРАУСКАЯ ОБЛАСТЬ',
       'ЗАПАДНО-КАЗАХСТАНСКАЯ ОБЛАСТЬ', 'ЖАМБЫЛСКАЯ ОБЛАСТЬ',
       'КАРАГАНДИНСКАЯ ОБЛАСТЬ', 'КОСТАНАЙСКАЯ ОБЛАСТЬ',
       'КЫЗЫЛОРДИНСКАЯ ОБЛАСТЬ', 'МАНГИСТАУСКАЯ ОБЛАСТЬ',
       'ЮЖНО-КАЗАХСТАНСКАЯ ОБЛАСТЬ', 'ПАВЛОДАРСКАЯ ОБЛАСТЬ',
       'СЕВЕРО-КАЗАХСТАНСКАЯ ОБЛАСТЬ', 'ВОСТОЧНО-КАЗАХСТАНСКАЯ ОБЛАСТЬ',
       'ГОРОД АСТАНА', 'ГОРОД АЛМАТЫ', 'ГОРОД ШЫМКЕНТ',
       'ТУРКЕСТАНСКАЯ ОБЛАСТЬ', 'ОБЛАСТЬ ЖЕТІСУ', 'ОБЛАСТЬ ҰЛЫТАУ',
       'ОБЛАСТЬ АБАЙ'], dtype=object)

In [21]:
# remove ЮКО
df = df[df["Регион"] != "ЮЖНО-КАЗАХСТАНСКАЯ ОБЛАСТЬ"]
df.sample(10)

,Показатель,Категория хозяйства,Регион,2015-01,2015-02,2015-03,2015-04,2015-05,2015-06,2015-07,...,2024-03,2024-04,2024-05,2024-06,2024-07,2024-08,2024-09,2024-10,2024-11,2024-12
41,КРС,Крестьянские или фермерские хозяйства,ОБЛАСТЬ ЖЕТІСУ,нет,нет,нет,нет,нет,нет,нет,...,4243.2,1249.1,233.9,3125.3,372.2,1019.6,2119.1,1768.8,2042.4,6647.4
366,Птицы,Крестьянские или фермерские хозяйства,ВОСТОЧНО-КАЗАХСТАНСКАЯ ОБЛАСТЬ,NaN,NaN,0.04,NaN,NaN,2.31,0.71,...,NaN,NaN,NaN,0.5,0.2,NaN,0.1,0.2,NaN,0.4
460,Молоко,Хозяйства населения,ОБЛАСТЬ ҰЛЫТАУ,нет,нет,нет,нет,нет,нет,нет,...,1584.9,519.7,1150.1,1086.9,3074,1261,1432.4,1234.9,1090.9,1990.1
278,Верблюды,Сельхозпредприятия,ВОСТОЧНО-КАЗАХСТАНСКАЯ ОБЛАСТЬ,0,0,0,1.14,1.22,1.2,0,...,NaN,0.8,NaN,NaN,0.5,NaN,NaN,0.4,NaN,0
133,Свиньи,Сельхозпредприятия,АКМОЛИНСКАЯ ОБЛАСТЬ,72.32,44.44,81.59,117.04,146.62,114.41,105.97,...,6.94,6.55,6.74,17.26,12.39,16.33,16.16,16.45,9.79,22.95
450,Молоко,Хозяйства населения,МАНГИСТАУСКАЯ ОБЛАСТЬ,68.1,80.3,86.6,92.9,593.5,454.4,454.4,...,0,0,0,0,0,0,0,0,0,0
26,КРС,Крестьянские или фермерские хозяйства,АТЫРАУСКАЯ ОБЛАСТЬ,343.15,145.05,325.72,264.21,203.92,269.1,198.13,...,377.1,468.5,346.1,569.7,370.1,352.8,487.1,665.2,617.4,1059.9
511,Яйца,Хозяйства населения,ЗАПАДНО-КАЗАХСТАНСКАЯ ОБЛАСТЬ,823,1459.9,3104.6,3964,6505.2,4427,4427,...,2413,3096,3923.5,4846.4,3177.4,3191.8,3263.8,1573.8,1730.9,1485.7
129,Овцы и козы,Хозяйства населения,ОБЛАСТЬ ЖЕТІСУ,нет,нет,нет,нет,нет,нет,нет,...,2077.3,656.4,500.8,3064.5,547.4,1156.3,2134.4,2237.4,2093.4,3738.4
462,Яйца,Сельхозпредприятия,РЕСПУБЛИКА КАЗАХСТАН,275458.3,254874.6,279812,288793.9,296930.1,291877,291877,...,329677.9,306714.6,303080.6,296788.2,306031,315839.7,315899.9,322745.3,321496,321106.4


In [22]:
# fill missing values with 0
df = df.fillna(0)

# replace "нет" with NA (Абай, Жетісу, Ұлытау, г.Шымкент, Туркестанская до их образования)
df.replace("нет", np.nan, inplace=True)
df.sample(10)

C:\Users\Acer\AppData\Local\Temp\ipykernel_15324\335651453.py:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.fillna(0)
C:\Users\Acer\AppData\Local\Temp\ipykernel_15324\335651453.py:5: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.replace("нет", np.nan, inplace=True)


,Показатель,Категория хозяйства,Регион,2015-01,2015-02,2015-03,2015-04,2015-05,2015-06,2015-07,...,2024-03,2024-04,2024-05,2024-06,2024-07,2024-08,2024-09,2024-10,2024-11,2024-12
520,Яйца,Хозяйства населения,ВОСТОЧНО-КАЗАХСТАНСКАЯ ОБЛАСТЬ,4250.10,5193.10,8476.20,9801.20,16476.10,13275.20,13275.20,...,3889.20,4720.30,6221.40,7206.20,5819.40,5475.60,5286.50,4241.70,3976.50,4110.70
318,Верблюды,Хозяйства населения,МАНГИСТАУСКАЯ ОБЛАСТЬ,152.72,105.34,75.90,36.80,135.24,181.70,28.52,...,21.20,59.30,79.10,222.20,99.40,99.40,559.80,209.80,251.20,353.30
14,КРС,Сельхозпредприятия,ВОСТОЧНО-КАЗАХСТАНСКАЯ ОБЛАСТЬ,159.16,156.45,148.85,163.80,144.58,239.64,114.96,...,104.42,97.76,255.84,135.62,90.49,92.79,157.31,135.47,105.11,140.81
173,Свиньи,Крестьянские или фермерские хозяйства,ОБЛАСТЬ ЖЕТІСУ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,38.80,0.00,0.00,9.60,2.40,2.20,9.80,4.20,11.20,11.60
119,Овцы и козы,Хозяйства населения,КЫЗЫЛОРДИНСКАЯ ОБЛАСТЬ,533.60,431.79,424.63,387.81,481.60,567.78,697.07,...,399.20,460.40,450.60,452.70,300.10,507.00,473.80,522.20,447.20,478.30
155,Свиньи,Крестьянские или фермерские хозяйства,АКМОЛИНСКАЯ ОБЛАСТЬ,12.26,11.78,23.18,44.75,44.31,111.57,110.17,...,72.70,10.80,8.20,14.50,14.30,19.90,12.80,17.50,13.90,24.10
433,Молоко,Крестьянские или фермерские хозяйства,ГОРОД АСТАНА,6.70,7.70,12.30,10.40,12.50,18.60,18.60,...,3.60,7.20,7.20,0.00,0.00,0.00,0.00,0.00,0.00,0.00
44,КРС,Хозяйства населения,РЕСПУБЛИКА КАЗАХСТАН,43997.42,41271.96,45765.33,34850.73,42898.31,60065.37,31661.72,...,32711.60,25615.80,32695.50,48320.00,22795.90,29969.90,48114.30,36211.00,40203.90,84110.20
160,Свиньи,Крестьянские или фермерские хозяйства,ЖАМБЫЛСКАЯ ОБЛАСТЬ,12.10,3.14,4.13,83.18,4.50,126.01,4.64,...,0.00,18.50,0.70,43.70,2.40,5.90,19.30,7.70,17.60,12.20
279,Верблюды,Сельхозпредприятия,ГОРОД АСТАНА,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [23]:
# check replacement
df.groupby("Регион")[df.columns[3:]].apply(lambda g: g.isna().sum()).reset_index()

,Регион,2015-01,2015-02,2015-03,2015-04,2015-05,2015-06,2015-07,2015-08,2015-09,...,2024-03,2024-04,2024-05,2024-06,2024-07,2024-08,2024-09,2024-10,2024-11,2024-12
0,АКМОЛИНСКАЯ ОБЛАСТЬ,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,АКТЮБИНСКАЯ ОБЛАСТЬ,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,АЛМАТИНСКАЯ ОБЛАСТЬ,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,АТЫРАУСКАЯ ОБЛАСТЬ,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,ВОСТОЧНО-КАЗАХСТАНСКАЯ ОБЛАСТЬ,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,ГОРОД АЛМАТЫ,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,ГОРОД АСТАНА,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,ГОРОД ШЫМКЕНТ,24,24,24,24,24,24,24,24,24,...,0,0,0,0,0,0,0,0,0,0
8,ЖАМБЫЛСКАЯ ОБЛАСТЬ,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,ЗАПАДНО-КАЗАХСТАНСКАЯ ОБЛАСТЬ,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 

In [24]:
# get the final data by summing across показатели & категория хозяйства
totals = df.groupby(["Показатель", "Регион"])[df.columns[3:]].agg(lambda x: x.sum() if x.notna().any() else np.nan).reset_index()
totals


,Показатель,Регион,2015-01,2015-02,2015-03,2015-04,2015-05,2015-06,2015-07,2015-08,...,2024-03,2024-04,2024-05,2024-06,2024-07,2024-08,2024-09,2024-10,2024-11,2024-12
0,Верблюды,АКМОЛИНСКАЯ ОБЛАСТЬ,0.00,0.00,0.40,0.00,0.00,0.00,0.09,1.00,...,0.0,0.0,0.00,0.0,0.00,0.0,0.00,0.00,0.0,0.00
1,Верблюды,АКТЮБИНСКАЯ ОБЛАСТЬ,101.98,67.47,374.84,115.59,218.72,14.15,19.77,3.00,...,391.0,126.1,229.10,12.2,24.10,5.7,49.87,63.00,154.3,538.40
2,Верблюды,АЛМАТИНСКАЯ ОБЛАСТЬ,1.00,0.20,51.60,25.40,0.00,61.90,76.87,95.57,...,12.5,16.4,11.60,16.7,15.70,24.0,7.70,6.90,5.4,12.60
3,Верблюды,АТЫРАУСКАЯ ОБЛАСТЬ,213.89,167.70,306.60,164.97,342.57,192.00,43.60,113.03,...,441.9,219.1,487.69,446.9,191.43,133.5,730.34,709.68,537.6,930.28
4,Верблюды,ВОСТОЧНО-КАЗАХСТАНСКАЯ ОБЛАСТЬ,0.30,0.00,0.00,1.14,6.12,4.70,4.20,1.80,...,0.0,0.8,0.00,0.0,0.50,0.0,0.00,0.40,0.0,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
163,Яйца,ОБЛАСТЬ ҰЛЫТАУ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1569.2,1156.4,1174.80,1957.4,1310.40,973.5,1757.90,961.60,806.3,1780.60
164,Яйца,ПАВЛОДАРСКАЯ ОБЛАСТЬ,9637.30,8361.40,9164.50,10447.60,14083.60,14200.20,14200.20,12827.60,...,16631.1,16747.1,17414.00,18832.4,18654.70,18689.8,18071.90,16535.50,15899.2,14834.90
165,Яйца,РЕСПУБЛИКА КАЗАХСТАН,325795.20,303234.00,376664.50,400243.30,462679.40,454045.70,454045.70,431318.00,...,381480.5,368147.0,377486.90,393946.9,397493.70,394871.5,385885.00,371208.00,362418.8,364206.50
166,Яйца,СЕВЕРО-КАЗАХСТАНСКАЯ ОБЛАСТЬ,37704.50,37570.60,41732.40,52925.70,68018.40,79389.60,79389.60,57759.90,...,49356.3,49099.2,51500.30,56233.7,58261.10,53762.5,51783.90,47959.40,47352.0,48441.50


### Merge with other indicators (температура, осадки, поголовье)

In [52]:
# open and read excel file
x_vars = pd.read_excel("C:/Users/Acer/Desktop/Madi/Tasks/Nowcasting/Data/Темпа-Осадки-Поголовье.xlsx", skiprows=1)
x_vars.head(5)

,Темпа,Unnamed: 1,1,2,3,4,5,6,7,8,...,3.9,4.9,5.9,6.9,7.9,8.9,9.9,10.9,11.9,12.9
0,Акмолинская область,NaN,-12.490323,-10.192857,-5.870968,4.490000,14.574194,20.763333,19.180645,16.529032,...,-4.809677,8.026667,10.696774,21.043333,20.270968,17.338710,11.300000,3.832258,-4.343333,-7.638710
1,Актюбинская область,NaN,-13.483871,-11.821429,-5.990323,6.356667,15.609677,25.126667,23.167742,20.732258,...,-4.800000,13.513333,13.083871,22.893333,22.912903,20.206452,14.083333,6.667742,-1.520000,-6.419355
2,Алматинская область,NaN,-5.432258,-1.100000,2.909677,12.126667,17.180645,20.980000,25.564516,22.100000,...,2.812903,11.100000,16.348387,22.023333,23.158065,23.487097,13.726667,9.222581,2.256667,-6.796774
3,Атырауская область,NaN,-6.222581,-6.300000,0.380645,10.856667,19.506452,28.906667,27.177419,25.241935,...,2.522581,17.776667,17.341935,27.453333,28.264516,25.774194,19.956667,10.648387,3.096667,-2.612903
4,Западно-Казахстанская область,NaN,-11.845161,-8.928571,-4.406452,7.470000,17.100000,25.283333,22.690323,20.348387,...,-4.632258,14.926667,13.332258,23.853333,24.132258,21.151613,15.246667,8.261290,0.423333,-5.496774


In [ ]:
# Step 2: Set header row (row 0) as actual column names
# x_vars.columns = x_vars.iloc[0]
# x_vars = x_vars.drop(index=0)
# x_vars
# Step 3: Rename first column as 'Регион' and melt the rest
x_vars = x_vars.rename(columns={x_vars.columns[0]: "Регион"})

# # # Step 4: Drop unnecessary columns if any (like the second unnamed column)
# x_vars = x_vars.drop(columns=[x_vars.columns[1]], errors='ignore')
# x_vars
# # # Step 5: Melt into long format
# df_long = x_vars.melt(id_vars="Регион", var_name="Месяц", value_name="Температура")
# df_long
# # # Step 6: Convert "Месяц" into proper period format
# # If "Месяц" values are integers (1 to 12) repeated across years:
# years = list(range(2015, 2025))  # adjust if needed
# months = list(range(1, 13)) * len(years)
# periods = pd.date_range(start="2015-01", periods=len(months), freq="M").strftime("%Y-%m")
# df_long["Период"] = periods[:len(df_long)]

# # Optional: drop original "Месяц"
# df_long = df_long.drop(columns="Месяц")

# # Step 7: Strip region names if needed
# df_long["Регион"] = df_long["Регион"].str.strip()

# # Preview
# print(df_long.head())


,Регион,Месяц,Температура
0,Акмолинская область,1,-12.490323
1,Актюбинская область,1,-13.483871
2,Алматинская область,1,-5.432258
3,Атырауская область,1,-6.222581
4,Западно-Казахстанская область,1,-11.845161
...,...,...,...
23155,Область Жетысу,12.9,685.000000
23156,Область Улытау,12.9,509.000000
23157,Область Абай,12.9,NaN
23158,ЮЖНО-КАЗАХСТАНСКАЯ ОБЛАСТЬ,12.9,NaN


In [ ]:
# drop unnecessary columns
x_vars = x_vars.drop(x_vars.columns[1], axis = 1)
x_vars

,Темпа,1,2,3,4,5,6,7,8,9,...,3.9,4.9,5.9,6.9,7.9,8.9,9.9,10.9,11.9,12.9
0,Акмолинская область,-12.490323,-10.192857,-5.870968,4.490000,14.574194,20.763333,19.180645,16.529032,11.496667,...,-4.809677,8.026667,10.696774,21.043333,20.270968,17.338710,11.300000,3.832258,-4.343333,-7.638710
1,Актюбинская область,-13.483871,-11.821429,-5.990323,6.356667,15.609677,25.126667,23.167742,20.732258,16.576667,...,-4.800000,13.513333,13.083871,22.893333,22.912903,20.206452,14.083333,6.667742,-1.520000,-6.419355
2,Алматинская область,-5.432258,-1.100000,2.909677,12.126667,17.180645,20.980000,25.564516,22.100000,14.036667,...,2.812903,11.100000,16.348387,22.023333,23.158065,23.487097,13.726667,9.222581,2.256667,-6.796774
3,Атырауская область,-6.222581,-6.300000,0.380645,10.856667,19.506452,28.906667,27.177419,25.241935,21.010000,...,2.522581,17.776667,17.341935,27.453333,28.264516,25.774194,19.956667,10.648387,3.096667,-2.612903
4,Западно-Казахстанская область,-11.845161,-8.928571,-4.406452,7.470000,17.100000,25.283333,22.690323,20.348387,17.446667,...,-4.632258,14.926667,13.332258,23.853333,24.132258,21.151613,15.246667,8.261290,0.423333,-5.496774
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
188,Область Жетысу,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1819.000000,1834.000000,1841.000000,1861.000000,747.000000,685.000000,685.000000,685.000000,685.000000,685.000000
189,Область Улытау,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,710.000000,711.000000,711.000000,747.000000,502.000000,510.000000,515.000000,515.000000,513.000000,509.000000
190,Область Абай,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,476.000000,444.000000,465.000000,495.000000,NaN,NaN,NaN,NaN,NaN,NaN
191,ЮЖНО-КАЗАХСТАНСКАЯ ОБЛАСТЬ,NaN,21618.000000,23462.000000,24485.000000,24982.000000,25244.000000,24192.000000,23703.000000,23424.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [39]:
# Flatten to "YYYY-MM" strings

# Generate full list of months
years = list(range(2015, 2025))  # 2015 to 2024
months = list(range(1, 13))      # 1 to 12

periods = [f"{year}-{str(month).zfill(2)}" for year in years for month in months]


In [40]:
periods

['2015-01',
 '2015-02',
 '2015-03',
 '2015-04',
 '2015-05',
 '2015-06',
 '2015-07',
 '2015-08',
 '2015-09',
 '2015-10',
 '2015-11',
 '2015-12',
 '2016-01',
 '2016-02',
 '2016-03',
 '2016-04',
 '2016-05',
 '2016-06',
 '2016-07',
 '2016-08',
 '2016-09',
 '2016-10',
 '2016-11',
 '2016-12',
 '2017-01',
 '2017-02',
 '2017-03',
 '2017-04',
 '2017-05',
 '2017-06',
 '2017-07',
 '2017-08',
 '2017-09',
 '2017-10',
 '2017-11',
 '2017-12',
 '2018-01',
 '2018-02',
 '2018-03',
 '2018-04',
 '2018-05',
 '2018-06',
 '2018-07',
 '2018-08',
 '2018-09',
 '2018-10',
 '2018-11',
 '2018-12',
 '2019-01',
 '2019-02',
 '2019-03',
 '2019-04',
 '2019-05',
 '2019-06',
 '2019-07',
 '2019-08',
 '2019-09',
 '2019-10',
 '2019-11',
 '2019-12',
 '2020-01',
 '2020-02',
 '2020-03',
 '2020-04',
 '2020-05',
 '2020-06',
 '2020-07',
 '2020-08',
 '2020-09',
 '2020-10',
 '2020-11',
 '2020-12',
 '2021-01',
 '2021-02',
 '2021-03',
 '2021-04',
 '2021-05',
 '2021-06',
 '2021-07',
 '2021-08',
 '2021-09',
 '2021-10',
 '2021-11',
 '20

In [34]:
# Set the first row as column labels
x_vars.columns = x_vars.iloc[0]  # Assign first row as header
x_vars
# df = df[1:].reset_index(drop=True)  # Drop the first row and reset index
# df.head(5)

,Темпа,NaN,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,...,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,11.0,12.0
0,Темпа,NaN,1.000000,2.000000,3.000000,4.000000,5.000000,6.000000,7.000000,8.000000,...,3.000000,4.000000,5.000000,6.000000,7.000000,8.000000,9.000000,10.000000,11.000000,12.000000
1,Акмолинская область,NaN,-12.490323,-10.192857,-5.870968,4.490000,14.574194,20.763333,19.180645,16.529032,...,-4.809677,8.026667,10.696774,21.043333,20.270968,17.338710,11.300000,3.832258,-4.343333,-7.638710
2,Актюбинская область,NaN,-13.483871,-11.821429,-5.990323,6.356667,15.609677,25.126667,23.167742,20.732258,...,-4.800000,13.513333,13.083871,22.893333,22.912903,20.206452,14.083333,6.667742,-1.520000,-6.419355
3,Алматинская область,NaN,-5.432258,-1.100000,2.909677,12.126667,17.180645,20.980000,25.564516,22.100000,...,2.812903,11.100000,16.348387,22.023333,23.158065,23.487097,13.726667,9.222581,2.256667,-6.796774
4,Атырауская область,NaN,-6.222581,-6.300000,0.380645,10.856667,19.506452,28.906667,27.177419,25.241935,...,2.522581,17.776667,17.341935,27.453333,28.264516,25.774194,19.956667,10.648387,3.096667,-2.612903
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
189,Область Жетысу,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1819.000000,1834.000000,1841.000000,1861.000000,747.000000,685.000000,685.000000,685.000000,685.000000,685.000000
190,Область Улытау,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,710.000000,711.000000,711.000000,747.000000,502.000000,510.000000,515.000000,515.000000,513.000000,509.000000
191,Область Абай,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,476.000000,444.000000,465.000000,495.000000,NaN,NaN,NaN,NaN,NaN,NaN
192,ЮЖНО-КАЗАХСТАНСКАЯ ОБЛАСТЬ,20408.0,NaN,21618.000000,23462.000000,24485.000000,24982.000000,25244.000000,24192.000000,23703.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [32]:
x_vars = x_vars.drop(index=df.index[:1]).reset_index(drop=True)
x_vars

,Unnamed: 0,Unnamed: 1,2015,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 112,Unnamed: 113,Unnamed: 114,Unnamed: 115,Unnamed: 116,Unnamed: 117,Unnamed: 118,Unnamed: 119,Unnamed: 120,Unnamed: 121
0,Акмолинская область,NaN,-12.490323,-10.192857,-5.870968,4.490000,14.574194,20.763333,19.180645,16.529032,...,-4.809677,8.026667,10.696774,21.043333,20.270968,17.338710,11.300000,3.832258,-4.343333,-7.638710
1,Актюбинская область,NaN,-13.483871,-11.821429,-5.990323,6.356667,15.609677,25.126667,23.167742,20.732258,...,-4.800000,13.513333,13.083871,22.893333,22.912903,20.206452,14.083333,6.667742,-1.520000,-6.419355
2,Алматинская область,NaN,-5.432258,-1.100000,2.909677,12.126667,17.180645,20.980000,25.564516,22.100000,...,2.812903,11.100000,16.348387,22.023333,23.158065,23.487097,13.726667,9.222581,2.256667,-6.796774
3,Атырауская область,NaN,-6.222581,-6.300000,0.380645,10.856667,19.506452,28.906667,27.177419,25.241935,...,2.522581,17.776667,17.341935,27.453333,28.264516,25.774194,19.956667,10.648387,3.096667,-2.612903
4,Западно-Казахстанская область,NaN,-11.845161,-8.928571,-4.406452,7.470000,17.100000,25.283333,22.690323,20.348387,...,-4.632258,14.926667,13.332258,23.853333,24.132258,21.151613,15.246667,8.261290,0.423333,-5.496774
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
188,Область Жетысу,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1819.000000,1834.000000,1841.000000,1861.000000,747.000000,685.000000,685.000000,685.000000,685.000000,685.000000
189,Область Улытау,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,710.000000,711.000000,711.000000,747.000000,502.000000,510.000000,515.000000,515.000000,513.000000,509.000000
190,Область Абай,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,476.000000,444.000000,465.000000,495.000000,NaN,NaN,NaN,NaN,NaN,NaN
191,ЮЖНО-КАЗАХСТАНСКАЯ ОБЛАСТЬ,20408.0,NaN,21618.000000,23462.000000,24485.000000,24982.000000,25244.000000,24192.000000,23703.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
# save into an excel file: все 8 показателей в сумме по категориям хозяйств в одном excel
totals.to_excel("Просуммированные по категориям.xlsx", index=False)

In [26]:
# save into excel files: 8 показателей в сумме по категориям хозяйств, каждый в отдельном excel

indicators = ["КРС", "Лошади", "Свиньи", "Овцы и козы", "Птицы", "Молоко", "Яйца", "Верблюды"]

for indicator in indicators:
    df_filtered = totals[totals["Показатель"] == indicator].drop(columns=["Показатель"])
    file_name = f"{indicator}_обработанные.xlsx"
    df_filtered.to_excel(file_name, index=False)
